# DATA 512 Assignment 2: Bias in Data

DATA 512 Fall 2018

Ryan Bae

Due: November 1st, 2018

In [1]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import requests
import json

In [3]:
# load datasets
page_data = pd.read_csv('page_data.csv')
wpds_2018 = pd.read_csv('WPDS_2018_data.csv')

Below is the function to call the Wikimedia ORES API to get the quality ratings of each article. The code below is taken from the course instructor Jonathan Morgan's github page in the link below:

https://github.com/Ironholds/data-512-a2/blob/master/hcds-a2-bias_demo.ipynb

In [4]:
# function to call ORES API. API limit is ~290, so the rev_ids must be split into smaller chunks.
headers = {'User-Agent' : 'https://github.com/ryanbae89',
           'From' : 'rbae@uw.edu'}

def get_ores_data(revision_ids, headers):
    
    # Define the endpoint
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'
    
    # Specify the parameters - smushing all the revision IDs together separated by | marks.
    # Yes, 'smush' is a technical term, trust me I'm a scientist.
    # What do you mean "but people trusting scientists regularly goes horribly wrong" who taught you tha- oh.  
    params = {'project' : 'enwiki',
              'model'   : 'wp10',
              'revids'  : '|'.join(str(x) for x in revision_ids)
              }
    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
#     print(json.dumps(response, indent=4, sort_keys=True))
    return response

In [ ]:
# get ORES scores for each article
# get_ores_data must be called in chunks due to API limits
rev_ids_full = list(page_data['rev_id'])
n = 100
rev_ids_chunks = [rev_ids_full[i:i+n] for i in range(0, len(rev_ids_full), n)]
ores_ratings = []

for rev_ids in rev_ids_chunks:
    ores_results = get_ores_data(rev_ids, headers)
    ores_ratings.append(ores_results['enwiki'])

In [ ]:
# turn to pandas dataframes and concatenate each chunk
ores = pd.DataFrame()
for ores_ratings_chunk in ores_ratings:
    ores_chunk = pd.DataFrame.from_dict(ores_ratings_chunk['scores'], orient='index')
    ores = pd.concat([ores, ores_chunk])
ores = ores.reset_index()
ores.columns = ['rev_id', 'score']

In [ ]:
# function to get the prediction
def get_pred(row):
    if 'score' in row.keys():
        return row['score']['prediction']
    else:
        return 'NaN'

# apply to every row in the ores dataframe
ores['prediction'] = ores['score'].apply(lambda x: get_pred(x))

In [ ]:
ores.head()